Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
import os
data_root = 'E:/MYGIT/DataSources/notMNIST'
pickle_file = os.path.join(data_root, 'notMNIST_remove_oevrlap.pickle')
pickle_file_1 = os.path.join(data_root, 'notMNIST.pickle')
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del(save)  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (8984, 28, 28) (8984,)
Test set (8719, 28, 28) (8719,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [11]:
labels = np.arange(5)
#labels = (np.arange(5) == labels[:,None]).astype(np.float32)
print(labels[:,None])
print(np.arange(5))

[[0]
 [1]
 [2]
 [3]
 [4]]
[0 1 2 3 4]


In [12]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (8984, 784) (8984, 10)
Test set (8719, 784) (8719, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [13]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
    ##以上步骤只是设置好了初始变量等参数，为后续运算做准备

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Let's run this computation and iterate:

In [33]:
a = np.array([[30,40,70],[80,20,10],[50,90,60]])  
print (np.argmax(a))
print (np.argmax(a,1)) 

7
[2 0 1]


In [37]:
ss = np.argmax(predictions, 1)
ss

array([6, 3, 2, 4, 3, 7, 9, 0, 7, 8, 3, 6, 7, 9, 3, 0, 7, 5, 3, 7, 3, 0,
       0, 2, 7, 3, 5, 8, 7, 2, 6, 5, 8, 7, 6, 6, 5, 9, 3, 2, 0, 4, 4, 0,
       3, 9, 9, 5, 4, 2, 1, 2, 5, 9, 0, 9, 0, 3, 0, 0, 2, 6, 1, 0, 8, 4,
       6, 5, 3, 8, 6, 1, 9, 6, 4, 2, 4, 6, 3, 4, 6, 3, 5, 5, 6, 6, 4, 5,
       8, 2, 4, 3, 8, 2, 2, 3, 4, 9, 1, 7, 8, 3, 2, 1, 8, 1, 9, 8, 0, 7,
       6, 7, 7, 5, 5, 5, 3, 8, 9, 6, 8, 0, 2, 9, 8, 6, 8, 6], dtype=int64)

In [17]:
num_steps = 801

def accuracy(predictions, labels):
    print()
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])    
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
            
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 19.905123
Training accuracy: 4.8%
Validation accuracy: 7.9%
Loss at step 100: 2.404097
Training accuracy: 71.0%
Validation accuracy: 68.8%
Loss at step 200: 1.952869
Training accuracy: 74.1%
Validation accuracy: 71.3%
Loss at step 300: 1.700572
Training accuracy: 75.3%
Validation accuracy: 72.3%
Loss at step 400: 1.530348
Training accuracy: 76.2%
Validation accuracy: 72.6%
Loss at step 500: 1.403522
Training accuracy: 77.0%
Validation accuracy: 72.8%
Loss at step 600: 1.303501
Training accuracy: 77.6%
Validation accuracy: 72.9%
Loss at step 700: 1.221707
Training accuracy: 78.1%
Validation accuracy: 73.2%
Loss at step 800: 1.153089
Training accuracy: 78.5%
Validation accuracy: 73.3%
Test accuracy: 80.9%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [29]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))##随机梯度参数设置不同之处
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))##随机梯度参数设置不同之处
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [30]:
num_steps = 3001
##3001*128 总共的训练数据   ## 之前是801*10000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        ##分母减去batch_size是防止当batch_size不能被训练数据量整除时，offset:(offset + batch_size)超出数组界限
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.270086
Minibatch accuracy: 11.7%
Validation accuracy: 13.1%
Minibatch loss at step 500: 0.996733
Minibatch accuracy: 78.1%
Validation accuracy: 74.3%
Minibatch loss at step 1000: 1.332731
Minibatch accuracy: 78.1%
Validation accuracy: 75.1%
Minibatch loss at step 1500: 1.465219
Minibatch accuracy: 75.0%
Validation accuracy: 76.1%
Minibatch loss at step 2000: 1.168050
Minibatch accuracy: 74.2%
Validation accuracy: 74.9%
Minibatch loss at step 2500: 0.585732
Minibatch accuracy: 86.7%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 0.954566
Minibatch accuracy: 79.7%
Validation accuracy: 76.8%
Minibatch loss at step 3500: 0.694876
Minibatch accuracy: 83.6%
Validation accuracy: 77.2%
Minibatch loss at step 4000: 0.778824
Minibatch accuracy: 75.8%
Validation accuracy: 77.5%
Minibatch loss at step 4500: 0.701170
Minibatch accuracy: 82.0%
Validation accuracy: 78.2%
Minibatch loss at step 5000: 0.855807
Minibatch accuracy: 77.3%
Validation accura

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [27]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))##隐藏层1024个节点
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.##设置神经网络tf.nn.relu()里的参数相当于逻辑回归的函数logits
    lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

    logits = tf.matmul(lay1_train, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1) 
    valid_prediction = tf.nn.softmax(tf.matmul(lay1_valid, weights2) + biases2)
    
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(lay1_test, weights2) + biases2)

In [28]:
num_steps = 3001
##3001*128 总共的训练数据   ## 之前是801*10000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 330.834229
Minibatch accuracy: 8.6%
Validation accuracy: 21.2%
Minibatch loss at step 500: 15.338503
Minibatch accuracy: 74.2%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 4.225451
Minibatch accuracy: 82.8%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 9.428771
Minibatch accuracy: 75.8%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 2.345299
Minibatch accuracy: 80.5%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 4.581788
Minibatch accuracy: 85.2%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 3.236370
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Minibatch loss at step 3500: 3.967705
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Minibatch loss at step 4000: 2.570257
Minibatch accuracy: 85.9%
Validation accuracy: 81.8%
Minibatch loss at step 4500: 4.254594
Minibatch accuracy: 87.5%
Validation accuracy: 80.6%
Minibatch loss at step 5000: 1.668830
Minibatch accuracy: 83.6%
Validation accur